## Mars vs Earth Seismology Comparison

In [1]:
# import necessary files for running code
import numpy as np
import matplotlib.pyplot as plt
from obspy import read_inventory, read
from obspy.clients.fdsn import Client
from obspy.core import UTCDateTime, Stream, read, Trace

### Mars Data

Location Info

In [2]:
def readSeismoData(stream, channel, freq):
    channel = channel
    freq=freq
    stream.filter(type='highpass', freq)
    stream.select(component='Z')
    trace = stream[0]

st1 = read("Mars.mseed")
st2 = read('S0931a_DISP.mseed')
st3 = read('S0931a_VEL.mseed')

channels = ['BHZ']

# stream 
st1.filter(type='highpass', freq=0.01)
st1 = st1.select(component='Z')
print(st1)  

#trace 
tr1 = st1[0]
print(tr1)
print(tr1.stats) 
print(tr1.data)

# stream 
st2.filter(type='highpass', freq=0.1)
st2 = st2.select(component='Z')
print(st2)  

#trace 
tr2 = st2[0]
print(tr2)
print(tr2.stats) 
print(tr2.data)

# stream 
st3.filter(type='highpass', freq=0.1)
st3 = st3.select(component='Z')
print(st3)  

#trace 
tr3 = st3[0]
print(tr3)
print(tr3.stats) 
print(tr3.data)

FileNotFoundError: [Errno 2] No such file or directory: 'S0931a_DISP.mseed'

### Earth Data

The data is taken in the Elysium Planetia region of the planet. Elysium Planetia is a region defined by high volcanic activity as result of a mantle plume with some of the highest slopes on the planet! Because of the mantle plume aspect, we decided to compare it to the Yellowstone region of the united states. The area is heavily monitered by the USGS and is centeralized by one large volcano just like Elysium Panetia (Elysium Mons).